In [2]:
#Import Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pprint
import json

#API Base URL
base_url = 'http://hotline.whalemuseum.org/api.json'
data = requests.get(base_url)
print(data)

# Pretty Print the output of the JSON
responses = requests.get(base_url).json()
print(json.dumps(responses, indent=4, sort_keys=True))

<Response [200]>
[
    {
        "created_at": "2017-11-09T18:16:30Z",
        "description": "Spotted from the Edmonds-Kingston ferry, near fishing boats.",
        "id": "5a049b7e686f743ec5040000",
        "latitude": 47.79953,
        "location": "",
        "longitude": -122.47996699999999,
        "orca_type": "unknown",
        "quantity": "5",
        "sighted_at": "2017-11-09T18:11:00Z",
        "species": "orca",
        "updated_at": "2017-11-14T22:20:45Z",
        "url": "http://hotline.whalemuseum.org/sightings/5a049b7e686f743ec5040000"
    },
    {
        "created_at": "2017-10-01T00:41:04Z",
        "description": "From the Inn at Langley looking east there was a very large group of porpoise swimming south in the Saratoga passage",
        "id": "59d039a0686f743ec5020000",
        "latitude": 48.047447813103005,
        "location": "Camano Island, WA, US",
        "longitude": -122.40477597314452,
        "orca_type": "unknown",
        "quantity": "20",
        "sighted

In [21]:
#Empty Lists to store data
lat = []
location = []
long = []
orca_type = []
quantity = []
species = []
count = 1

# Initialize loop
for response in responses:
    lat.append(response['latitude'])
    long.append(response['longitude'])
    location.append(response['location'])
    species.append(response['species'])
    quantity.append(response['quantity'])
    orca_type.append(response['orca_type'])
    count = count + 1

orca_dict = {
    'Species': species,
    'Orca Type': orca_type,
    'Quantity': quantity,
    'Location': location,
    'Latitude': lat,
    'Longitude': long

}
# Convert dictionary to dataframe
orca_data = pd.DataFrame.from_dict(orca_dict)
orca_data.head()

,Species,Orca Type,Quantity,Location,Latitude,Longitude
0,orca,unknown,5,,47.799530,-122.479967
1,harbor porpoise,unknown,20,"Camano Island, WA, US",48.047448,-122.404776
2,humpback,unknown,1,"Gig Harbor, WA, US",47.307289,-122.561674
3,humpback,unknown,1,"Steilacoom, WA, US",47.166960,-122.614546
4,humpback,unknown,1,"Edmonds, WA, US",47.884579,-122.342120


In [20]:
#Reading CSV for whale data
filename = "resources\seamap_csv.csv"
seamap_data = pd.read_csv(filename, header=0, index_col=0)
#seamap_data
seamap_data.reset_index(drop=True, inplace=True)
seamap_data.drop(['oid', 'datasets', 'geom_wkt', 'csquare'], axis = "columns", inplace=True)
seamap_data


,longitude,latitude,providers,species,date_min,date_max,num_records,num_animals,num_species
0,-179,-72,"David Bowden, NIWA;Rochelle Constantine, Unive...",Balaenoptera bonaerensis,2/26/2018,2/26/2018,2,4.0,1
1,-179,-71,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,2/28/2018,3/1/2018,3,6.0,1
2,-179,-69,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,3/2/2018,3/2/2018,1,2.0,1
3,-179,-68,"David Bowden, NIWA;Rochelle Constantine, Unive...",Balaenoptera acutorostrata;Megaptera novaeangliae,12/23/1910,3/6/2018,6,24.0,2
4,-179,-67,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,3/6/2018,3/6/2018,5,11.0,1
...,...,...,...,...,...,...,...,...,...
16138,180,62,National Marine Mammal Laboratory;Wildlife Con...,Balaena mysticetus;Balaenoptera acutorostrata;...,6/1/1913,7/31/2010,10,21.0,4
16139,180,64,Russian Cetacean Habitat Project via Happywhal...,Balaena mysticetus;Megaptera novaeangliae,6/1/1913,8/3/2005,18,18.0,2
16140,180,65,Wildlife Conservation Society,Balaena mysticetus,6/1/1913,7/1/1913,11,11.0,1
16141,180,65,Wildlife Conservation Society,Balaena mysticetus,6/1/1913,6/1/1913,1,1.0,1
